<a href="https://colab.research.google.com/github/vochiphat/DS-Projects/blob/main/Dataframe_and_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1 Pandas Dataframe

In [14]:
!pip -q install pyspark==3.5.1


In [4]:
import pandas as pd


#Data
data = [["Children", "First", 6, 0],["Children", "Second", 24, 0],
    ["Children", "Third", 27, 52],
    ["Men", "First", 57, 118],
    ["Men", "Second", 14, 154],
    ["Men", "Third", 75, 387],
    ["Men", "Crew", 192, 693],
    ["Women", "First", 140, 4],
    ["Women", "Second", 80, 13],
    ["Women", "Third", 76, 89],
    ["Women", "Crew", 20, 3]
]


In [5]:
#Create dataframe from data
titanic = pd.DataFrame( data = data,  columns = ["sex", "class", "survived", "died"])

In [6]:
#Display top 10 rows
titanic.head(10)

,sex,class,survived,died
0,Children,First,6,0
1,Children,Second,24,0
2,Children,Third,27,52
3,Men,First,57,118
4,Men,Second,14,154
5,Men,Third,75,387
6,Men,Crew,192,693
7,Women,First,140,4
8,Women,Second,80,13
9,Women,Third,76,89


In [7]:
#statistical description of the dataframe
titanic.describe()

,survived,died
count,11.000000,11.000000
mean,64.636364,137.545455
std,57.874472,216.880319
min,6.000000,0.000000
25%,22.000000,3.500000
50%,57.000000,52.000000
75%,78.000000,136.000000
max,192.000000,693.000000


In [8]:
#remove crew member data
titanic = titanic.drop(titanic[titanic['class'] =='Crew'].index)
titanic

,sex,class,survived,died
0,Children,First,6,0
1,Children,Second,24,0
2,Children,Third,27,52
3,Men,First,57,118
4,Men,Second,14,154
5,Men,Third,75,387
7,Women,First,140,4
8,Women,Second,80,13
9,Women,Third,76,89


In [10]:
#Total people for each group
titanic["total_passenger"] = (titanic['survived'] + titanic['died'])
titanic

,sex,class,survived,died,total_passenger
0,Children,First,6,0,6
1,Children,Second,24,0,24
2,Children,Third,27,52,79
3,Men,First,57,118,175
4,Men,Second,14,154,168
5,Men,Third,75,387,462
7,Women,First,140,4,144
8,Women,Second,80,13,93
9,Women,Third,76,89,165


In [11]:
#Survival rate column
titanic['survival_rate %'] = (titanic['survived'] / titanic['total_passenger'] *100).map("{:.2f}".format).astype(float)
titanic

,sex,class,survived,died,total_passenger,survival_rate %
0,Children,First,6,0,6,100.00
1,Children,Second,24,0,24,100.00
2,Children,Third,27,52,79,34.18
3,Men,First,57,118,175,32.57
4,Men,Second,14,154,168,8.33
5,Men,Third,75,387,462,16.23
7,Women,First,140,4,144,97.22
8,Women,Second,80,13,93,86.02
9,Women,Third,76,89,165,46.06


In [12]:
#drop total people column
titanic = titanic.drop('total_passenger', axis = 1)
titanic

,sex,class,survived,died,survival_rate %
0,Children,First,6,0,100.00
1,Children,Second,24,0,100.00
2,Children,Third,27,52,34.18
3,Men,First,57,118,32.57
4,Men,Second,14,154,8.33
5,Men,Third,75,387,16.23
7,Women,First,140,4,97.22
8,Women,Second,80,13,86.02
9,Women,Third,76,89,46.06


In [13]:
#filter survival rate > 80%
titanic = titanic[titanic['survival_rate %'] > 80]
titanic

,sex,class,survived,died,survival_rate %
0,Children,First,6,0,100.00
1,Children,Second,24,0,100.00
7,Women,First,140,4,97.22
8,Women,Second,80,13,86.02


## Part 2 Pyspark

In [2]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.getOrCreate()

#Data
titanic = spark.createDataFrame(
    [("Children","First",6,0),("Children","Second",24,0),("Children","Third",27,52),
     ("Men","First",57,118),("Men","Second",14,154),("Men","Third",75,387),("Men","Crew",192,693),
     ("Women","First",140,4),("Women","Second",80,13),("Women","Third",76,89),("Women","Crew",20,3)],
    ["sex","class","survived","died"]
)
titanic.show(100, truncate=False)

#Survival rate
titanic_with_rate = titanic.withColumn(
    "survival_rate_pct", F.round(F.col("survived")/(F.col("survived")+F.col("died"))*100, 2)
)
#Filter SR >80%
titanic_with_rate.filter("survival_rate_pct > 80").show(truncate=False)


+--------+------+--------+----+
|sex     |class |survived|died|
+--------+------+--------+----+
|Children|First |6       |0   |
|Children|Second|24      |0   |
|Children|Third |27      |52  |
|Men     |First |57      |118 |
|Men     |Second|14      |154 |
|Men     |Third |75      |387 |
|Men     |Crew  |192     |693 |
|Women   |First |140     |4   |
|Women   |Second|80      |13  |
|Women   |Third |76      |89  |
|Women   |Crew  |20      |3   |
+--------+------+--------+----+

+--------+------+--------+----+-----------------+
|sex     |class |survived|died|survival_rate_pct|
+--------+------+--------+----+-----------------+
|Children|First |6       |0   |100.0            |
|Children|Second|24      |0   |100.0            |
|Women   |First |140     |4   |97.22            |
|Women   |Second|80      |13  |86.02            |
|Women   |Crew  |20      |3   |86.96            |
+--------+------+--------+----+-----------------+

